# This notebook uses Unsloth to do QLoRA with Llama 3.1 8B Instruct on Colab.  

info:  
https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing  

https://colab.research.google.com/drive/135ced7oHytdxu3N2DNe1Z0kqjyYIkDXp?usp=sharing  

https://docs.unsloth.ai/tutorials/how-to-finetune-llama-3-and-export-to-ollama  

https://github.com/unslothai/unsloth/wiki  

https://huggingface.co/docs/transformers/index  

### Install unsloth and Hagging Face's datasets module

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-sb127_bs/unsloth_5b9fd6f5ff4746a5817e0bd49ed26082
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-sb127_bs/unsloth_5b9fd6f5ff4746a5817e0bd49ed26082
  Resolved https://github.com/unslothai/unsloth.git to commit 52bc19d1fa4cd3557b785127fd68b5f4d1c34347
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 7.9 MB/s eta 0:00:00
   ━━

### Set the parameters you need to test/choose.  

Info:  
https://huggingface.co/docs/trl/v0.7.4/en/sft_trainer#trl.SFTTrainer https://huggingface.co/docs/transformers/v4.35.0/en/main_classes/trainer#transformers.TrainingArguments

In [2]:
# Set these input parameters

# Dataset parameters
age_range = '4_6' # Choose '1_3' or '4_6'

# Model paramters
context_length = 4096 #context length of 2048 roughly equals 1500 words.

### Import the packages used throughout

In [3]:
import torch
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### Load a Large Language Model

In [4]:
# Model info: https://huggingface.co/unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit
max_seq_length = context_length
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    dtype = None,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

## Run this cell only when you want to save the base model without the LoRA adaptor.   

- I'm not sure if the Q4KM GGUF file uses the same quantization as the model loaded in the RAM, that is, the model used in the following LoRA training. So, make sure to check the performance by yourself. Later, we will merge the base model and the trained LoRA adaptor then save the merged model.

- This file conversion takes over 30 min with T4 on Colab.  

At the end, there will be the following files in the "base_model" directory:  

-rw-r--r-- 1 root root  977 Aug 15 22:36 config.json  
-rw-r--r-- 1 root root  234 Aug 15 22:36 generation_config.json  
-rw-r--r-- 1 root root 4.7G Aug 15 22:38 pytorch_model-00001-of-00004.bin  
-rw-r--r-- 1 root root 4.7G Aug 15 22:39 pytorch_model-00002-of-00004.bin  
-rw-r--r-- 1 root root 4.6G Aug 15 22:40 pytorch_model-00003-of-00004.bin  
-rw-r--r-- 1 root root 1.1G Aug 15 22:40 pytorch_model-00004-of-00004.bin  
-rw-r--r-- 1 root root  24K Aug 15 22:40 pytorch_model.bin.index.json  
-rw-r--r-- 1 root root  454 Aug 15 22:36 special_tokens_map.json  
-rw-r--r-- 1 root root  55K Aug 15 22:36 tokenizer_config.json  
-rw-r--r-- 1 root root 8.7M Aug 15 22:36 tokenizer.json  
-rw-r--r-- 1 root root  15G Aug 15 22:46 unsloth.F16.gguf  
-rw-r--r-- 1 root root 4.6G Aug 15 23:03 unsloth.Q4_K_M.gguf  

If you run this cell you will copy only unsloth.Q4_K_M.gguf to your Google Drive.  

In [ ]:
if False:
  # Mount your Google Drive
  from google.colab import drive
  drive.mount('/content/drive')

  # Put the base model data to a GGUF file. Quantize with Q4km.
  model.save_pretrained_gguf("base_model", tokenizer, quantization_method = "q4_k_m")

  # Now copy the GGUF file for the qunatized model data to your Google drive:
  !cp /content/base_model/unsloth.Q4_K_M.gguf /content/drive/MyDrive/
  !rm -r ./base_model # Delete to free the disk memory

### Define the prompt used for training and testing  

This is the chat format for Llama 3.1 when the "ipython" (tools) is not used.  
Info: https://llama.meta.com/docs/model-cards-and-prompt-formats/llama3_1/


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023  
Today Date: 12 Aug 2024

{}<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}


In [5]:
llama_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 12 Aug 2024

{}<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}"""

#llama_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
#
#{}<|eot_id|><|start_header_id|>user<|end_header_id|>
#
#{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
#
#{}"""

We will use this prompt for our LoRA training as well as for testing.

In [6]:
if age_range == '1_3':
  input_prompt = llama_prompt.format(
                    'You are a creative assistant who writes stories for children. Your goal is to write a story having surprising twists and a happy ending. At each of your chat turns, write either a chapter whose length is less than 1000 words or an entire story whose length is less than 1000 words. In addition, generate a title of the chapter or story. Furthermore, at the beginning of your chat turn, please indicate whether your writing is meant to be a story chapter or an entire story by saying either "chapter title:" or "story title:" in front of the title you produce.', # instruction
                    'Write a story for one-year-old children, two-year-old children, or three-year-old children.', # input
                    '' # output - leave this blank for generation!
                    )
elif age_range == '4_6':
  input_prompt = llama_prompt.format(
                    'You are a creative assistant who writes stories for children. Your goal is to write a story having surprising twists and a happy ending. At each of your chat turns, write either a chapter whose length is less than 1000 words or an entire story whose length is less than 1000 words. In addition, generate a title of the chapter or story. Furthermore, at the beginning of your chat turn, please indicate whether your writing is meant to be a story chapter or an entire story by saying either "chapter title:" or "story title:" in front of the title you produce.', # instruction
                    'Write a story for four-year-old children, five-year-old children, or six-year-old children.', # input
                    '' # output - leave this blank for generation!
                    )


In [7]:
print(input_prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 12 Aug 2024

You are a creative assistant who writes stories for children. Your goal is to write a story having surprising twists and a happy ending. At each of your chat turns, write either a chapter whose length is less than 1000 words or an entire story whose length is less than 1000 words. In addition, generate a title of the chapter or story. Furthermore, at the beginning of your chat turn, please indicate whether your writing is meant to be a story chapter or an entire story by saying either "chapter title:" or "story title:" in front of the title you produce.<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a story for four-year-old children, five-year-old children, or six-year-old children.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




### Inference without LoRA  
We can further improve the output by choosing more parameters explained here:  

NoBadWordsLogitsProcessor  
https://huggingface.co/docs/transformers/v4.44.0/en/internal/generation_utils#transformers.NoBadWordsLogitsProcessor

Stopping criteria  
https://huggingface.co/docs/transformers/v4.44.0/en/internal/generation_utils#transformers.StoppingCriteria

transformers.GenerationConfig  
https://huggingface.co/docs/transformers/v4.44.0/en/main_classes/text_generation#transformers.GenerationConfig



In [8]:
FastLanguageModel.for_inference(model)
inputs = tokenizer([input_prompt], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 12 Aug 2024

You are a creative assistant who writes stories for children. Your goal is to write a story having surprising twists and a happy ending. At each of your chat turns, write either a chapter whose length is less than 1000 words or an entire story whose length is less than 1000 words. In addition, generate a title of the chapter or story. Furthermore, at the beginning of your chat turn, please indicate whether your writing is meant to be a story chapter or an entire story by saying either "chapter title:" or "story title:" in front of the title you produce.<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a story for four-year-old children, five-year-old children, or six-year-old children.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

story title: Benny's Magical Garden Adventure

Once upon a time, in a small village surrounded by gr

Second identical run to check if the model reproduces the same story

In [9]:
FastLanguageModel.for_inference(model)
inputs = tokenizer([input_prompt], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 12 Aug 2024

You are a creative assistant who writes stories for children. Your goal is to write a story having surprising twists and a happy ending. At each of your chat turns, write either a chapter whose length is less than 1000 words or an entire story whose length is less than 1000 words. In addition, generate a title of the chapter or story. Furthermore, at the beginning of your chat turn, please indicate whether your writing is meant to be a story chapter or an entire story by saying either "chapter title:" or "story title:" in front of the title you produce.<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a story for four-year-old children, five-year-old children, or six-year-old children.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

story title: Benny's Magical Paintbrush

Once upon a time, in a small village surrounded by rolling 

### Prepare the dataset  
More info about chat templates:
https://docs.unsloth.ai/basics/chat-templates

In [10]:
from datasets import Dataset

# Create a dataset defined in gen_age_1_3.py or gen_age_4_6.py.
# Info: https://huggingface.co/docs/datasets/v2.20.0/en/create_dataset
# This dataset has three features: system, user, assistant.
if age_range == '1_3':
    from gen_age_1_3 import gen_age_1_3
    dataset = Dataset.from_generator(gen_age_1_3)
elif age_range == '4_6':
    from gen_age_4_6 import gen_age_4_6
    dataset = Dataset.from_generator(gen_age_4_6)

# For the QLoRA training, create a chain of Llama chat conversations from the dataset.
# Take the data from dataset and put it to llama_prompt.
# This function adds another feature called 'text' to the dataset.
EOS_TOKEN = tokenizer.eos_token # End of sentense token
def formatting_prompts_func(examples):
    instructions = examples["system"]
    inputs       = examples["user"]
    outputs      = examples["assistant"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = llama_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [11]:
print(dataset['text'])

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 12 Aug 2024\n\nYou are a creative assistant who writes stories for children. Your goal is to write a story having surprising twists and a happy ending. At each of your chat turns, write either a chapter whose length is less than 1000 words or an entire story whose length is less than 1000 words. In addition, generate a title of the chapter or story. Furthermore, at the beginning of your chat turn, please indicate whether your writing is meant to be a story chapter or an entire story by saying either "chapter title:" or "story title:" in front of the title you produce.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWrite a story for four-year-old, five-year-old, or six-year-old children.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nstory title: Where the Wild Things Are\n\nThe night Max wore his wolf suit and made mischief of one kind, and another. His m

## LoRA Training

In [27]:
# LoRA parameters
lora_r = 2
lora_alpha = 4
#lora_target_modules = ["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"]
lora_target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

# Training parameters
num_train_epochs = 6
learning_rate = 8e-4 #2e-4 to 8e-4 gave a smooth decrease of the training loss.
per_device_train_batch_size = 1
gradient_accumulation_steps = 8 #7 for '4_6', 8 for '4_6'
warmup_steps = 3
logging_steps = 1

# **************** 4_6
# lora_target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
# learning_rate = 8e-4, lora_r = 2, lora_alpha = 4
# (  6 epochs, Training loss 2.3 --> 1.9, warmup_steps = 3) may be close to the right level.
# ( 10 epochs, Training loss 2.3 --> 1.7, warmup_steps = 5) produced stories very similar to what's available online.
# ( 20 epochs, Training loss 2.3 --> 1.2, warmup_steps = 5) cited stories existing online, but not in the dataset.




# **************** 1_3
# ****** With lora_target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
# With learning_rate = 4e-4, lora_r = 2, lora_alpha = 4,
# ( 5 epochs, Training loss 1.0) was underfit, poor performance...warmup_steps = 1
# ( 8 epochs, Training loss 0.3) was underfit but sometime overfit.
# ( 9 epochs, Training loss 1.3-1.9) was underfit.
# (10 epochs, Training loss 0.6-0.8) was close to the right training level. <-- Might be the best so far. Generally good, but produced a copy of the train story as well sometimes
# (12 epochs, Training loss 1.8) was underfit.
# (15 epochs, Training loss 0.07) was close to the right training level.
# (15 epochs, Training loss 1.6) was underfit.
# (20 epochs, Training loss 1.3) was underfit.
# (30 epochs, Training loss 0.9) was mixed result, a bit underfit.

# So maybe, try learning_rate = 8e-4, lora_r = 2, lora_alpha = 4,
# To speed up the training and aim to get Training loss = 0.6-0.8

# With learning_rate = 4e-4, lora_r = 3, lora_alpha = 6,
# (10 epochs, Training loss ) was
# (20 epochs, Training loss ) was
# (30 epochs, Training loss ) was



# ******** With , lora_target_modules = ["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"]
# With learning_rate = 4e-4, lora_r = 1, lora_alpha = 2
# (10 epochs, Training loss 1.8) was underfit.
# (15 epochs, Training loss 1.4) was underfit.
# (20 epochs, Training loss 1.1) was poor, produced only the title
# (25 epochs, Training loss 0.9) was overfit.

# With learning_rate = 4e-4, lora_r = 1, lora_alpha = 4
# (15 epochs, Training loss 1.1) was poor, produced only the title


# With learning_rate = 2e-4, lora_r = 2, lora_alpha = 2
# (20 epochs, Training loss 1.5) was somewhat underfit.
# (25 epochs, Training loss 1.3) was somewhat underfit.
# (27 epochs, Training loss 1.2) was optimal for this set of parameters, but perhaps the performance is not impressive.
# (30 epochs, Training loss 1.1) was sometimes good, some times bad, also cited Noh's arch.
# (40 epochs, Training loss 0.9) was a overfit often.

# With learning_rate = 4e-4, lora_r = 2, lora_alpha = 2
# (10 epochs, Training loss 1.7) was underfit.
# (15 epochs, Training loss 1.3) was close to the right training level.
# (20 epochs, Training loss 1.0) was a clear overfit.

# With learning_rate = 4e-4, lora_r = 2, lora_alpha = 4
# ( 8 epochs, Training loss 1.6) was slightly underfit, not good performance.
# (10 epochs, Training loss 1.5) was close to the right training level, but slightly underfit.
# (12 epochs, Training loss 1.3) was overfit.
# (15 epochs, Training loss 1.0) was a clear overfit.

# With learning_rate = 4e-4, lora_r = 8, lora_alpha = 16
# (3 epochs, Training loss 1.9) was very poor ....warmup_steps = 1
# (6 epochs, Training loss 1.2) was clear overfit.

# With learning_rate = 2e-4, lora_r = 8, lora_alpha = 8
# (5 epochs, Training loss 1.8) was underfit ....warmup_steps = 1
# (7 epochs, Training loss 1.6) was underfit ....warmup_steps = 1
# (10 epochs, Training loss ) was poor ..mostly produced only the title



#### Define LoRA configuration  
info:  
peft.LoraConfig  
https://huggingface.co/docs/peft/v0.12.0/en/package_reference/lora#peft.LoraConfig  

peft.get_peft_model  
https://huggingface.co/docs/peft/v0.12.0/en/package_reference/peft_model#peft.get_peft_model

In [28]:
merged_model = FastLanguageModel.get_peft_model(
    model,
    r = lora_r,
    lora_alpha = lora_alpha,
    target_modules = lora_target_modules,
    use_rslora=True, # rank stabilized LoRA
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    random_state = 3407,
    loftq_config = None, # LoftQ
)

#### Train LoRA  
Info:  
https://huggingface.co/docs/trl/v0.7.4/en/sft_trainer#trl.SFTTrainer
https://huggingface.co/docs/transformers/v4.35.0/en/main_classes/trainer#transformers.TrainingArguments

In [29]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = merged_model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        num_train_epochs = num_train_epochs,
        per_device_train_batch_size = per_device_train_batch_size,
        gradient_accumulation_steps = gradient_accumulation_steps,
        warmup_steps = warmup_steps,
        learning_rate = learning_rate,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = logging_steps,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 8 | Num Epochs = 6
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 6
 "-____-"     Number of trainable parameters = 1,703,936


Step,Training Loss
1,2.323500
2,2.323500
3,2.268200
4,2.150500
5,2.006800
6,1.928800


### Inference with LoRA

We can further improve the output by choosing more parameters explained here:  

NoBadWordsLogitsProcessor  
https://huggingface.co/docs/transformers/v4.44.0/en/internal/generation_utils#transformers.NoBadWordsLogitsProcessor

Stopping criteria  
https://huggingface.co/docs/transformers/v4.44.0/en/internal/generation_utils#transformers.StoppingCriteria

transformers.GenerationConfig  
https://huggingface.co/docs/transformers/v4.44.0/en/main_classes/text_generation#transformers.GenerationConfig

In [31]:
story_char_number = 0
while story_char_number < 500:
  FastLanguageModel.for_inference(merged_model)
  inputs = tokenizer([input_prompt], return_tensors = "pt").to("cuda")

  outputs_tensor = merged_model.generate(**inputs, max_new_tokens = 2048, use_cache = True) # Output tensor
  output_text = tokenizer.batch_decode(outputs_tensor) # Output list of length 1.
  output_text = output_text[0] # Convert from list to str
  output_text = output_text.replace(input_prompt, '') # Get rid of the input prompt
  output_text = output_text.replace('<|begin_of_text|>', '') # Get rid of '<|begin_of_text|>'
  story_char_number = len(output_text) # Count the number of the charactors in the generated story

print(output_text)

story title: "The Little Rabbit's Big Adventure"

Once upon a time, in a sunny meadow, a little rabbit named Rosie lived in a cozy burrow with her family. Rosie loved to play outside, chasing butterflies and sniffing wildflowers. One day, while exploring, Rosie stumbled upon a beautiful, shimmering rainbow-colored door hidden behind a bush.

The door was locked, but Rosie was curious. She tried to open it, but it wouldn't budge. Just as she was about to give up, a friendly old owl perched on a nearby tree called out, "Hello, little rabbit! What are you doing?"

Rosie explained that she wanted to see what was behind the door. The owl replied, "Ah, that's the entrance to the Magic Garden. But it's not for rabbits. You need a special key to open it."

Rosie asked, "What kind of key?" The owl said, "A key made of kindness, friendship, and love. You see, the garden is a place where flowers bloom only when people show kindness to one another."

Rosie thought for a moment and then asked, "How

In [ ]:
if False:
  # Use this to generate the text with TextStreamer
  FastLanguageModel.for_inference(merged_model)
  inputs = tokenizer([input_prompt], return_tensors = "pt").to("cuda")

  text_streamer = TextStreamer(tokenizer)
  _ = merged_model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

### Save the trained model

In [ ]:
# This saves only the LoRA adapter.
merged_model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

if False:
  # This saves the LoRA-and-base-model mmerged and q4_k_m-quantized model in GGUF file.
  merged_model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
  #merged_model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now copy the model data to Google drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!ls -lh ./lora_model

In [ ]:
!cp -r ./lora_model ./drive/MyDrive/